In [1]:
import cv2
import dlib
import imutils
import torch
from scipy.spatial import distance
from imutils import face_utils
from playsound import playsound
import pygame
from ultralytics import YOLO

c:\Users\Jaka Putra Pratama\AppData\Local\Programs\Python\Python310\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Fungsi EAR
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [3]:
# Threshold dan frame check
thresh = 0.25
frame_check = 20
flag = 0
alarm_on = False

In [4]:
# Inisialisasi detektor dlib
face_detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]

In [5]:
phone_model = YOLO('mobile_phone_training/phone_detector5/weights/best.pt')

In [ ]:
# Mulai webcam
cap = cv2.VideoCapture(1)

In [7]:
#  VARIABEL PENTING UNTUK PHONE DETECTION
# Variabel phone detection (pastikan sudah ada)
if 'phone_detection_frames' not in globals():
    phone_detection_frames = 0
if 'phone_confirmation_threshold' not in globals():
    phone_confirmation_threshold = 3  # Turunkan ke 3 frame

print("✅ Phone detection variables initialized!")
print(f"📊 Confirmation threshold: {phone_confirmation_threshold} frames")
print(f"📊 Current phone frames: {phone_detection_frames}")

✅ Phone detection variables initialized!
📊 Confirmation threshold: 3 frames
📊 Current phone frames: 0


In [8]:
pygame.mixer.init()
try:
    pygame.mixer.music.load("alarm.wav")
except:
    print("⚠️ alarm.wav not found")

print("🔧 CLEAN DETECTION MODE")
print("📱 Only showing valid phone detections")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = imutils.resize(frame, width=640)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = face_detector(gray, 0)

    # ===== DROWSINESS DETECTION =====
    drowsy_detected = False
    
    for subject in subjects:
        shape = predictor(gray, subject)
        shape = face_utils.shape_to_np(shape)
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0

        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

        if ear < thresh:
            flag += 1
        else:
            flag = 0
        
        if flag >= frame_check:
            drowsy_detected = True
        
        cv2.putText(frame, f"EAR: {ear:.3f}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # ===== CLEAN PHONE DETECTION (NO YELLOW BOX) =====
    frame_height, frame_width = frame.shape[:2]
    
    # YOLO detection
    results = phone_model(frame, conf=0.3, iou=0.4, verbose=False)
    
    valid_phone_detections = 0

    for result in results:
        boxes = result.boxes
        if boxes is not None:
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                class_name = phone_model.names[cls]
                
                # Filter parameters (relaxed)
                box_width = x2 - x1
                box_height = y2 - y1
                
                # Size filter
                min_width = frame_width * 0.05
                max_width = frame_width * 0.5
                min_height = frame_height * 0.08
                max_height = frame_height * 0.6
                
                # Aspect ratio filter
                aspect_ratio = box_height / box_width if box_width > 0 else 0
                
                # Position filter
                center_y = (y1 + y2) / 2
                
                # Apply filters
                size_ok = (min_width <= box_width <= max_width and 
                          min_height <= box_height <= max_height)
                ratio_ok = 1.0 <= aspect_ratio <= 4.0
                position_ok = center_y > frame_height * 0.2
                confidence_ok = conf > 0.3
                
                # Class filter
                phone_keywords = ['phone', 'cell', 'mobile', 'smartphone']
                class_ok = any(keyword in class_name.lower() for keyword in phone_keywords)
                
                # HANYA TAMPILKAN DETECTION YANG VALID (NO YELLOW BOX)
                if confidence_ok and size_ok and ratio_ok and position_ok and class_ok:
                    valid_phone_detections += 1
                    
                    # Draw HANYA valid detection (RED BOX)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
                    cv2.putText(frame, f"📱 Phone: {conf:.1%}", 
                               (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.5, (0, 0, 255), 2)
                    cv2.putText(frame, f"Valid Detection", 
                               (x1, y2+15), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.4, (0, 0, 255), 1)

    # ===== CONFIRMATION LOGIC =====
    if valid_phone_detections > 0:
        phone_detection_frames += 1
    else:
        phone_detection_frames = max(0, phone_detection_frames - 1)

    # Phone confirmed
    phone_confirmed = phone_detection_frames >= phone_confirmation_threshold

    # ===== DISPLAY INFO (CLEAN) =====
    cv2.putText(frame, f"Phone Confirmation: {phone_detection_frames}/{phone_confirmation_threshold}", 
               (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
    cv2.putText(frame, f"Drowsy Frames: {flag}/{frame_check}", 
               (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

    # ===== ALERT SYSTEM =====
    if drowsy_detected or phone_confirmed:
        if phone_confirmed and drowsy_detected:
            alert_msg = "🚨 CRITICAL: DROWSY + PHONE"
        elif phone_confirmed:
            alert_msg = "📱 PHONE CONFIRMED"
        elif drowsy_detected:
            alert_msg = "😴 DROWSINESS DETECTED"
        
        cv2.putText(frame, alert_msg, (10, 110),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "****************ALERT!****************", (10, 140),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        if not alarm_on:
            try:
                pygame.mixer.music.play(-1)
                alarm_on = True
            except:
                print("🔊 ALERT!")
    else:
        if alarm_on:
            pygame.mixer.music.stop()
            alarm_on = False

    # ===== STATUS DISPLAY =====
    if phone_confirmed:
        status = "📱 PHONE CONFIRMED"
        color = (0, 0, 255)
    elif phone_detection_frames > 0:
        status = f"📱 DETECTING ({phone_detection_frames}/{phone_confirmation_threshold})"
        color = (0, 165, 255)
    else:
        status = "✅ NO PHONE"
        color = (0, 255, 0)

    cv2.putText(frame, status, (10, frame.shape[0] - 20),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    # ===== DISPLAY FRAME =====
    cv2.imshow("Clean Phone Detection", frame)
    
    # ===== KEYBOARD CONTROLS =====
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        breakq
    elif key == ord("r"):
        phone_detection_frames = 0
        flag = 0
        print("🔄 Reset counters")
    elif key == ord("1"):
        phone_confirmation_threshold = 1
        print("🎛️ Ultra sensitive: 1 frame")
    elif key == ord("2"):
        phone_confirmation_threshold = 3
        print("🎛️ Sensitive: 3 frames")
    elif key == ord("3"):
        phone_confirmation_threshold = 5
        print("🎛️ Balanced: 5 frames")
    elif key == ord("c"):
        # Adjust confidence threshold
        print("Current confidence: 0.3")
        print("Press 'h' for higher (0.5) or 'l' for lower (0.2)")
    elif key == ord("h"):
        print("🎛️ Higher confidence mode (0.5)")
        # Code will use 0.5 confidence in next iteration
    elif key == ord("l"):
        print("🎛️ Lower confidence mode (0.2)")
        # Code will use 0.2 confidence in next iteration

cap.release()
cv2.destroyAllWindows()


🔧 CLEAN DETECTION MODE
📱 Only showing valid phone detections


NameError: name 'breakq' is not defined